In [1]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from ssl import create_default_context
import requests
from getpass import getpass
import pandas as pd
import numpy as np
import json
from datetime import datetime
from datetime import timedelta
import parametros
import random
import re

## Conectando a ElasticSearch

La ultima línea se utiliza para garantizar la ejecución de la consulta
* timeout es el tiempo para cada ejecución
* max_retries el número de intentos si la conexión falla
* retry_on_timeout para activar los reitentos

In [2]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=60, max_retries=3, retry_on_timeout=True
) 

### Calculando fechas para la ejecución

* Se calculan las fechas para asociar al nombre del indice
* fecha_hoy, es usada para concatenar al nombre del indice principal previa inserción
* ahora, es la fecha en un formato requerido para ES

In [3]:
now = datetime.now()
format_ES = "%Y.%m.%d"
fecha_hoy = str(now.strftime(format_ES))
ahora_format = "%Y-%m-%d"'T'"%H:%M:%S"
ahora = str(now.strftime(ahora_format))

### Definiendo indice principal con fecha de hoy

In [4]:
indice = parametros.gestion_tablero15_index 
indice_control = parametros.tableros_mintic_control

### Función para generar JSON compatible con ES

In [5]:
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

### Trae la ultima fecha para control de ejecución

Cuando en el rango de tiempo de la ejecución, no se insertan nuevos valores, las fecha maxima en indice mintic no aumenta, por tanto se usa esta fecha de control para garantizar que incremente el bucle de ejecución

In [6]:
total_docs = 1
try:
    response = es.search(
        index= indice_control,
        body={
           "_source": ["tablero15.fechaControl"],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia-tablero15"
                    }
                  }
              ]
            }
          }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion = doc["_source"]['tablero15.fechaControl']
except Exception as e:
    print(e)
    response["hits"]["hits"] = []
if response["hits"]["hits"] == []:
    fecha_ejecucion = '2021-05-14T00:00:00'
print("ultima fecha para control de ejecucion:",fecha_ejecucion)



ultima fecha para control de ejecucion: 2021-08-14T22:10:00


### leyendo indice semilla-inventario

En el script que ingesta semilla, trae la información de los centros de conexión administrados. Para el indice principal se requiere:

* site_id como llave del centro de conexión.
* Datos geográficos (Departamento, municipio, centro poblado, sede.)

In [7]:
total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source": ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob','energiadesc'
                           ,'nombreSede','latitud','longitud','id_Beneficiario','COD_ISO','codDanesede',
                           'cod_servicio','codDaneMuni','nombre_centro_pob','codCentroPoblado','codDaneInstitucionEdu',
                           'tipoSitio','detalleSitio','energia','region','matricula','DDA','grupoDesc','estadoInstalacion',
                           'nombreInstitucionEd']
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_semilla = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
except Exception as e:
    print(e)
    print("fecha:",now,"- Error en lectura de datos semilla")
    #exit()

In [8]:
def get_location(x,y='lat'):
    patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrón que debe cumplir
    if (not patron.match(x) is None) and (str(x)!=''):
        return x.replace(',','.')
    else:
        #Código a ejecutar si las coordenadas no son válidas
        return '4.596389' if y=='lat' else '-74.074639'
    
datos_semilla['latitud'] = datos_semilla['latitud'].apply(lambda x:get_location(x,'lat'))
datos_semilla['longitud'] = datos_semilla['longitud'].apply(lambda x:get_location(x,'lon'))
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["longitud"]=='a') | (datos_semilla["latitud"]=='a')].index)
datos_semilla['gestion.tablero15.location'] = datos_semilla['latitud'] + ',' + datos_semilla['longitud']
datos_semilla['gestion.tablero15.location']=datos_semilla['gestion.tablero15.location'].str.replace('a,a','')
datos_semilla.drop(columns=['latitud','longitud'],inplace=True)


In [9]:
datos_semilla = datos_semilla.rename(columns={'nombreSede':'gestion.tablero15.nombreSede'
                                               , 'DDA':'gestion.tablero15.DDA'
                                               , 'estadoInstalacion':'gestion.tablero15.estadoInstalacion' 
                                               , 'COD_ISO':'gestion.tablero15.COD_ISO'
                                               , 'energia':'gestion.tablero15.energia'
                                               , 'nombre_departamento':'gestion.tablero15.departamento'
                                               , 'codCentroPoblado':'gestion.tablero15.codCentroPoblado'
                                               , 'nombreInstitucionEd':'gestion.tablero15.nombreInstitucionEd'
                                               , 'codDanesede':'gestion.tablero15.codDanesede'
                                               , 'tipoSitio':'gestion.tablero15.tipoSitio'
                                               , 'codDaneMuni':'gestion.tablero15.codDaneMuni'
                                               , 'nombre_centro_pob':'gestion.tablero15.nombre_centro_pob'
                                               , 'site_id':'gestion.tablero15.site_id'
                                               , 'matricula':'gestion.tablero15.matricula'
                                               , 'energiadesc':'gestion.tablero15.energiadesc'
                                               , 'grupoDesc':'gestion.tablero15.grupoDesc'
                                               , 'cod_servicio':'gestion.tablero15.cod_servicio'
                                               , 'region':'gestion.tablero15.region'
                                               , 'detalleSitio':'gestion.tablero15.detalleSitio'
                                               , 'nombre_municipio':'gestion.tablero15.municipio'
                                               , 'id_Beneficiario':'gestion.tablero15.id_Beneficiario'
                                               , 'codDaneInstitucionEdu':'gestion.tablero15.codDaneInstitucionEdu'
                                           })        


In [10]:
 datos_semilla

,gestion.tablero15.nombreSede,gestion.tablero15.DDA,gestion.tablero15.estadoInstalacion,gestion.tablero15.COD_ISO,gestion.tablero15.energia,gestion.tablero15.departamento,gestion.tablero15.codCentroPoblado,gestion.tablero15.nombreInstitucionEd,gestion.tablero15.codDanesede,gestion.tablero15.tipoSitio,...,gestion.tablero15.matricula,gestion.tablero15.energiadesc,gestion.tablero15.grupoDesc,gestion.tablero15.cod_servicio,gestion.tablero15.region,gestion.tablero15.detalleSitio,gestion.tablero15.municipio,gestion.tablero15.id_Beneficiario,gestion.tablero15.codDaneInstitucionEdu,gestion.tablero15.location
0,C.E.R LA PAULINA,5,3,CO-ANT,1,ANTIOQUIA,333,C. E. R. LA PAULINA,205854000501,1,...,111,RED INTERCONECTADA,OBLIGATORIOS,ZGYO176,A,8,VALDIVIA,22154,205854000501,"6.7048548,-76.14525469"
1,CENTRO EDUCATIVO RURAL EL CERRO,5,3,CO-ANT,1,ANTIOQUIA,333,I. E. MANUEL ANTONIO TORO,205284000201,1,...,22,RED INTERCONECTADA,OBLIGATORIOS,ZGYO177,A,8,FRONTINO,20022,105284000214,"4.225166667,-73.10716667"
2,C. E. R. MERCEDES YEPES,5,1,CO-ANT,1,ANTIOQUIA,333,C. E. R. EL CARMIN,205040000161,1,...,42,RED INTERCONECTADA,OBLIGATORIOS,,A,8,ANORÍ,18787,205040000357,"6.93848274,-75.151177190"
3,C. E. R. MEDIAS FALDAS,5,1,CO-ANT,4,ANTIOQUIA,333,I. E. ANORI,205040000616,1,...,84,TIENE ENERGÍA,Opción 2,,A,8,ANORÍ,18802,105040000212,"7.04797951,-75.19442611"
4,C. E. R. ROSAURA ORTEGA,5,2,CO-ANT,1,ANTIOQUIA,333,C. E. R. EL CARMIN,205040000641,1,...,14,RED INTERCONECTADA,OBLIGATORIOS,ZGYO583,A,8,ANORÍ,18804,205040000357,"7.06272584,-75.22730972"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6942,ESCUELA SANTA ISABEL,5,1,CO-VAU,3,VAUPÉS,333,I.E.D. DE PIEDRA ÑI,297001002239,1,...,53,SOLUCIÓN SOLAR,OBLIGATORIOS,,A,8,PACOA,67733,297001001879,"6.39026335,-73.138349330"
6943,ESCUELA RURAL DE SAN LUIS DE PIEDRA ÑI,5,1,CO-VAU,4,VAUPÉS,333,I.E.D. DE PIEDRA ÑI,297511000036,1,...,14,SIN INFORMACIÓN,OBLIGATORIOS,,A,8,PACOA,70622,297001001879,"0.08164385,-70.07416384"
6944,ESCUELA RURAL DE SANTA ROSA,5,1,CO-VAU,4,VAUPÉS,333,I.E.D. DE PIEDRA ÑI,297511000044,1,...,36,SIN INFORMACIÓN,OBLIGATORIOS,,A,8,PACOA,70623,297001001879,"4.596389,-74.074639"
6945,ESCUELA RURAL DE TOAKA,5,1,CO-VAU,4,VAUPÉS,333,I.E.D. DE PIEDRA ÑI,297511000028,1,...,32,SIN INFORMACIÓN,OBLIGATORIOS,,A,8,PACOA,70624,297001001879,"1.007644476,-71.25103723"


In [11]:
 datos_semilla.dtypes

gestion.tablero15.nombreSede               object
gestion.tablero15.DDA                       int64
gestion.tablero15.estadoInstalacion         int64
gestion.tablero15.COD_ISO                  object
gestion.tablero15.energia                   int64
gestion.tablero15.departamento             object
gestion.tablero15.codCentroPoblado          int64
gestion.tablero15.nombreInstitucionEd      object
gestion.tablero15.codDanesede               int64
gestion.tablero15.tipoSitio                 int64
gestion.tablero15.codDaneMuni               int64
gestion.tablero15.nombre_centro_pob        object
gestion.tablero15.site_id                  object
gestion.tablero15.matricula                object
gestion.tablero15.energiadesc              object
gestion.tablero15.grupoDesc                object
gestion.tablero15.cod_servicio             object
gestion.tablero15.region                   object
gestion.tablero15.detalleSitio              int64
gestion.tablero15.municipio                object


### Leyendo indice servicemanager-interacciones

Se lee la información de los De Service Manager. En la lectura se traen todas las interacciones reportadas por servicemanager, con la información de la misma.

* id_beneficiario, es la llave para cruzar con cada centro de conexión.
* variable1 y variable2, son la tipo y subtipo de la interaccion.
* current_phase, estado del ticket
* clr_txt_assignment, es el sujeto asignado al ticket
* owner_name, usuario del ticket
* open_time y close_time, son las fecha de apertura y cierre del ticket
* resolution, respuesta al ticket
* incident_id, numero del ticket
* source, fuente del ticket(email o telefono)
* severity, gravedad del ticket
* clr_txt_codcategorymtc categoria segun mintic

In [29]:
def traeSMInteracciones(fecha_max,fecha_tope):
    total_docs = 10000
    response = es.search(
        index= parametros.sm_interaciones,
        body={
            "_source": ['problem_status',
    'status',
    'variable1',
    'sysmodtime',
    'incident_id',
    'from_source',
    'clr_bmcdatevent',
    'severity',
    'clr_bmc_host',
    'assignment',
    'clr_bmc_location',
    'number',
    'product_type',
    'resolution'
                  , 'opened_by'
                  , 'subcategory'
                  , '@timestamp'
                  , 'open_time'
                  , 'clr_txt_idbneficmtc'
                  , 'category'
                  , 'close_time'
                  , 'source'
                  , 'key_char'
                  , 'total'
                  , 'close_date'
                  , 'resolution_code'
                  , 'resolved_time'
                  , 'callback_type'
                  , 'initial_impact'
                  , 'name'
                  , 'variable2'
                  , '@version'
                  , 'contact_name'
                  , 'variable3'
                        ]
            ,"query": {
              "range": {
                "open_time": {
                    "gte": fecha_max,
                    "lt": fecha_tope
                }
              }
          }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_SM_interactions = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))
    if datos_SM_interactions is None or datos_SM_interactions.empty:
        return pd.DataFrame()
    
    datos_SM_interactions = datos_SM_interactions.rename(columns={'variable1':'gestion.tablero15.categoria'
                                                                  , 'opened_by':'gestion.tablero15.opened_by'
                                                                  , 'subcategory':'gestion.tablero15.subcategoria'
                                                                  , '@timestamp':'gestion.tablero15.@timestamp'
                                                                  , 'open_time':'gestion.tablero15.fechaApertura'
                                                                  , 'clr_txt_idbneficmtc':'gestion.tablero15.id_Beneficiario'
                                                                  , 'category':'gestion.tablero15.category'
                                                                  , 'close_time':'gestion.tablero15.fechaCierre'
                                                                  , 'source':'gestion.tablero15.canal'
                                                                  , 'key_char':'gestion.tablero15.numeroTicket'
                                                                  , 'total':'gestion.tablero15.total'
                                                                  , 'close_date':'gestion.tablero15.close_date'
                                                                  , 'resolution_code':'gestion.tablero15.coderesolucion'
                                                                  , 'resolved_time':'gestion.tablero15.timeresolution'
                                                                  , 'callback_type':'gestion.tablero15.callback_type'
                                                                  , 'initial_impact':'gestion.tablero15.initial_impact'
                                                                  , 'name':'gestion.tablero15.name'
                                                                  , 'variable2':'gestion.tablero15.variable2'
                                                                  , '@version':'gestion.tablero15.version'
                                                                  , 'contact_name':'gestion.tablero15.contact_name'                                                                  
                                                                  , 'variable3':'gestion.tablero15.variable3'})         
    datos_SM_interactions = datos_SM_interactions.dropna(subset=['gestion.tablero15.id_Beneficiario'])
    datos_SM_interactions = datos_SM_interactions.drop(datos_SM_interactions[datos_SM_interactions['gestion.tablero15.id_Beneficiario']=="null"].index)
    datos_SM_interactions['gestion.tablero15.id_Beneficiario'] = datos_SM_interactions['gestion.tablero15.id_Beneficiario'].astype(int)
    datos_SM_interactions['gestion.tablero15.canal'] = datos_SM_interactions['gestion.tablero15.canal'].replace(['5','6'],['Correo Electrónico','Teléfono'])
    return datos_SM_interactions

Realizando bucle hasta conseguir datos de servicemanager-interacciones o hasta la fecha actual para realizar la carga de datos 

In [30]:
fecha_max_mintic = fecha_ejecucion

fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)-timedelta(seconds=1)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
datos_SM_interactions = traeSMInteracciones(fecha_max_mintic,fecha_tope_mintic)


if datos_SM_interactions is None or datos_SM_interactions.empty:
    while (datos_SM_interactions is None or datos_SM_interactions.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d"'T'"%H:%M:%S")) < str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
        datos_SM_interactions = traeSMInteracciones(fecha_max_mintic,fecha_tope_mintic)
else:
    pass

Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFr

In [14]:
datos_SM_interactions

""


In [15]:
datos_SM_interactions.columns

Index([], dtype='object')

# Leyendo indice servicemanager-incidentes

Se lee la información de los De Service Manager. En la lectura se traen todas las interacciones reportadas por servicemanager, con la información de la misma.

clr_bmc_location, es la llave para cruzar con cada centro de conexión.
product_type, es el detalle del ticket.
clr_bmc_host, estado la IP
assignment, es el sujeto asignado al ticket
contact_name, usuario del ticket
open_time y resolved_time, son las fecha de apertura y cierre del ticket
resolution, respuesta al ticket
number, numero del ticket
source, fuente del ticket(email o telefono)
severity, gravedad del ticket
category y subcategory, clasificacion dada por mintic

* Se consulta SM incidentes, que tengan fecha de resolucion dentro del rango procesado
* la duración de la falla es la diferencia en horas de resolved_time y open_time, menos los tiempos de parada
* el id del incidente en el indice incidentes es number, en interacciones es incident_id

In [16]:
def traeSMIncidentes(fecha_max,fecha_tope):
    print(fecha_max)
    print(fecha_tope)
    total_docs = 10000
    try:
        response = es.search(
            index= parametros.sm_incidentes,
            body={
                "_source": ['problem_status',
    'status',
    'variable1',
    'sysmodtime',
    'incident_id',
    'from_source',
    'clr_bmcdatevent',
    'severity',
    'clr_bmc_host',
    'assignment',
    'clr_bmc_location',
    'number',
    'product_type',
    'resolution'
                  , 'opened_by'
                  , 'subcategory'
                  , '@timestamp'
                  , 'open_time'
                  , 'clr_txt_idbneficmtc'
                  , 'category'
                  , 'close_time'
                  , 'source'
                  , 'key_char'
                  , 'total'
                  , 'close_date'
                  , 'resolution_code'
                  , 'resolved_time'
                  , 'callback_type'
                  , 'initial_impact'
                  , 'name'
                  , 'variable2'
                  , '@version'
                  , 'contact_name'
                  , 'variable3']
                ,"query": {
                    "range": {
                      "@timestamp": {
                        "gte": fecha_max,
                        "lt": fecha_tope
                      }
                    }
                }
            },
            size=total_docs
        )
        elastic_docs = response["hits"]["hits"]
        fields = {}
        for num, doc in enumerate(elastic_docs):
            source_data = doc["_source"]
            for key, val in source_data.items():
                try:
                    fields[key] = np.append(fields[key], val)
                except KeyError:
                    fields[key] = np.array([val])
                    
        datos_SM_incidents = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))
        
        if not datos_SM_incidents.empty:
            datos_SM_incidents['gestion.categoria'] = 'Dano' 
            datos_SM_incidents['gestion.subcategoria'] = 'Dano masivo internet' 
            datos_SM_incidents = datos_SM_incidents.rename(columns={'severity':'gestion.gravedad',
                                                                    'product_type' : 'gestion.detallesTicket',
                                                                    'contact_name':'gestion.usuarioTicket',
                                                                    'assignment':'gestion.responsable',
                                                                    'clr_bmc_location':'site_id',
                                                                    'clr_bmc_host':'gestion.IP',
                                                                    'number':'gestion.numeroTicket'})


            datos_SM_incidents['gestion.tiempoRespuesta'] = datos_SM_incidents.apply(lambda row: datetime.strptime(row.resolved_time, "%Y-%m-%d"'T'"%H:%M:%S") - datetime.strptime(row.open_time, "%Y-%m-%d"'T'"%H:%M:%S") if( row.resolved_time is not None) else 0, axis=1)
            if not datos_SM_incidents.empty:
                datos_SM_incidents['gestion.gravedad'] = datos_SM_incidents['gestion.gravedad'].replace(['1','2','3'],['Alto','Medio','Bajo'])
                datos_SM_incidents['gestion.duracion'] = datos_SM_incidents.apply(lambda row: datetime.strptime(row.resolved_time, "%Y-%m-%d"'T'"%H:%M:%S") - datetime.strptime(row.open_time, "%Y-%m-%d"'T'"%H:%M:%S") if( row.resolved_time is not None) else 0, axis=1)
            '''
            ### Por si no sirve ese calculo de horas
            aaa['f1'] = [datetime.strptime(x, "%Y-%m-%d %H:%M:%S") for x in concat["gestion.fechaApertura"]]
            aaa['f2'] = [datetime.strptime(x, "%Y-%m-%d %H:%M:%S") for x in concat["gestion.fechaControl"]]
            aaa['f3'] = ((aaa['f1']-aaa['f2']).dt.seconds)/3600
            '''
        return datos_SM_incidents
    except Exception as e:
        print("error")
        print(e)
        return pd.DataFrame()
    

In [17]:
fecha_max_mintic = fecha_ejecucion
fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)-timedelta(seconds=1)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
datos_SM_incidents = traeSMIncidentes(fecha_max_mintic,fecha_tope_mintic)

if datos_SM_incidents is None or datos_SM_incidents.empty:
    while (datos_SM_incidents is None or datos_SM_incidents.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d"'T'"%H:%M:%S")) < str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
        datos_SM_incidents = traeSMIncidentes(fecha_max_mintic,fecha_tope_mintic)
else:
    pass
#datos_SM_incidents

2021-08-14T22:10:00
2021-08-14T22:19:59
2021-08-14T22:20:00
2021-08-14T22:29:59


In [18]:
datos_SM_incidents

,sysmodtime,clr_txt_company_code,open_time,resolution,gestion.numeroTicket,incident_id,opened_by,@version,problem_status,gestion.gravedad,...,gestion.IP,clr_txt_idbneficmtc,subcategory,category,initial_impact,status,gestion.categoria,gestion.subcategoria,gestion.tiempoRespuesta,gestion.duracion
0,2021-07-07T14:33:25,,2021-06-30T16:42:33,Diagnosed Successfully,inci.number,SD1427690,smservicedeskplus,1,,Alto,...,,45827,Falla Conectividad MINTIC,incident,3,,Dano,Dano masivo internet,"36 days, 16:59:09","36 days, 16:59:09"
1,2021-07-07T17:07:50,,2021-06-08T14:14:04,Diagnosed Successfully,inci.number,SD1405357,smservicedeskplus,1,,Alto,...,,39335,Falla Conectividad MINTIC,incident,3,,Dano,Dano masivo internet,"55 days, 22:04:53","55 days, 22:04:53"
2,2021-07-08T14:26:38,,2021-06-08T16:34:04,Diagnosed Successfully,inci.number,SD1405671,smservicedeskplus,1,,Alto,...,,38799,Falla Conectividad MINTIC,incident,3,,Dano,Dano masivo internet,"55 days, 22:51:03","55 days, 22:51:03"
3,2021-07-17T16:31:12,,2021-07-02T16:12:40,Resolved Successfully,inci.number,SD1430401,smservicedeskplus,1,,Alto,...,,30353,Falla Conectividad MINTIC,incident,3,,Dano,Dano masivo internet,"15 days, 0:13:30","15 days, 0:13:30"
4,2021-07-09T22:28:49,,2021-06-30T18:48:29,Request Rejected,inci.number,SD1427849,smservicedeskplus,1,,Alto,...,,32087,Falla Conectividad MINTIC,incident,3,,Dano,Dano masivo internet,"38 days, 6:16:09","38 days, 6:16:09"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2021-08-11T15:49:12,,2021-07-13T17:27:22,None,inci.number,SD1441030,Adriel Fernando Aldornoz,1,,Bajo,...,,None,Falla Conectividad MINTIC,incident,2,,Dano,Dano masivo internet,0,0
150,2021-08-11T15:22:05,,2021-07-13T17:26:21,None,inci.number,SD1441027,Adriel Fernando Aldornoz,1,,Alto,...,,None,Falla Conectividad MINTIC,incident,1,,Dano,Dano masivo internet,0,0
151,2021-08-11T16:24:02,,2021-07-13T17:41:52,None,inci.number,SD1441054,Adriel Fernando Aldornoz,1,,Medio,...,,None,Falla Conectividad MINTIC,incident,3,,Dano,Dano masivo internet,0,0
152,2021-08-12T09:12:52,,2021-07-13T17:47:26,None,inci.number,SD1441067,Adriel Fernando Aldornoz,1,,Medio,...,,None,Falla Conectividad MINTIC,incident,3,,Dano,Dano masivo internet,0,0


In [19]:
datos_SM_incidents = datos_SM_incidents.rename(columns={'callback_type':'gestion.callback_type'
                                                                    , 'problem_status':'gestion.problem_status'
                                                                    , 'status':'gestion.status'
                                                                    , 'initial_impact':'gestion.initial_impact'
                                                                    , 'close_time':'gestion.close_time'
                                                                    , 'sysmodtime':'gestion.sysmodtime'
                                                                    , 'incident_id':'gestion.incident_id'
                                                                    , 'from_source':'gestion.from_source'
                                                                    , 'opened_by':'gestion.opened_by'
                                                                    , 'clr_bmcdatevent':'gestion.clr_bmcdatevent'
                                                                    , 'severity':'gestion.gravedad'
                                                                    , 'subcategory':'gestion.subcategory'
                                                                    , 'clr_bmc_host':'gestion.IP'
                                                                    , 'clr_txt_idbneficmtc':'gestion.id_Beneficiario'
                                                                    , 'assignment':'gestion.responsable'
                                                                    , 'category':'gestion.category'
                                                                    , 'contact_name':'gestion.usuarioTicket'
                                                                    , 'clr_bmc_location':'site_id'
                                                                    , 'number':'gestion.numeroTicket'
                                                                    , '@version':'gestion.@version'
                                                                    , 'clr_txt_company_code':'gestion.clr_txt_company_code'
                                                                    , 'open_time':'gestion.open_time'
                                                                    , 'product_type':'gestion.detallesTicket'
                                                                    , 'resolution':'gestion.resolution'
                                                                    , 'resolved_time':'gestion.resolved_time'
                                                                    , '@timestamp':'gestion.@timestamp'})

In [20]:
datos_SM_incidents

,gestion.sysmodtime,gestion.clr_txt_company_code,gestion.open_time,gestion.resolution,gestion.numeroTicket,gestion.incident_id,gestion.opened_by,gestion.@version,gestion.problem_status,gestion.gravedad,...,gestion.IP,gestion.id_Beneficiario,gestion.subcategory,gestion.category,gestion.initial_impact,gestion.status,gestion.categoria,gestion.subcategoria,gestion.tiempoRespuesta,gestion.duracion
0,2021-07-07T14:33:25,,2021-06-30T16:42:33,Diagnosed Successfully,inci.number,SD1427690,smservicedeskplus,1,,Alto,...,,45827,Falla Conectividad MINTIC,incident,3,,Dano,Dano masivo internet,"36 days, 16:59:09","36 days, 16:59:09"
1,2021-07-07T17:07:50,,2021-06-08T14:14:04,Diagnosed Successfully,inci.number,SD1405357,smservicedeskplus,1,,Alto,...,,39335,Falla Conectividad MINTIC,incident,3,,Dano,Dano masivo internet,"55 days, 22:04:53","55 days, 22:04:53"
2,2021-07-08T14:26:38,,2021-06-08T16:34:04,Diagnosed Successfully,inci.number,SD1405671,smservicedeskplus,1,,Alto,...,,38799,Falla Conectividad MINTIC,incident,3,,Dano,Dano masivo internet,"55 days, 22:51:03","55 days, 22:51:03"
3,2021-07-17T16:31:12,,2021-07-02T16:12:40,Resolved Successfully,inci.number,SD1430401,smservicedeskplus,1,,Alto,...,,30353,Falla Conectividad MINTIC,incident,3,,Dano,Dano masivo internet,"15 days, 0:13:30","15 days, 0:13:30"
4,2021-07-09T22:28:49,,2021-06-30T18:48:29,Request Rejected,inci.number,SD1427849,smservicedeskplus,1,,Alto,...,,32087,Falla Conectividad MINTIC,incident,3,,Dano,Dano masivo internet,"38 days, 6:16:09","38 days, 6:16:09"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2021-08-11T15:49:12,,2021-07-13T17:27:22,None,inci.number,SD1441030,Adriel Fernando Aldornoz,1,,Bajo,...,,None,Falla Conectividad MINTIC,incident,2,,Dano,Dano masivo internet,0,0
150,2021-08-11T15:22:05,,2021-07-13T17:26:21,None,inci.number,SD1441027,Adriel Fernando Aldornoz,1,,Alto,...,,None,Falla Conectividad MINTIC,incident,1,,Dano,Dano masivo internet,0,0
151,2021-08-11T16:24:02,,2021-07-13T17:41:52,None,inci.number,SD1441054,Adriel Fernando Aldornoz,1,,Medio,...,,None,Falla Conectividad MINTIC,incident,3,,Dano,Dano masivo internet,0,0
152,2021-08-12T09:12:52,,2021-07-13T17:47:26,None,inci.number,SD1441067,Adriel Fernando Aldornoz,1,,Medio,...,,None,Falla Conectividad MINTIC,incident,3,,Dano,Dano masivo internet,0,0


## Haciendo merge entre semilla e interacciones

Realizando inserción

In [21]:
use_these_keys = [ 'gestion.tablero15.nombreSede'
                    , 'gestion.tablero15.DDA'
                    , 'gestion.tablero15.estadoInstalacion' 
                    , 'gestion.tablero15.COD_ISO'
                    , 'gestion.tablero15.energia'
                    , 'gestion.tablero15.departamento'
                    , 'gestion.tablero15.codCentroPoblado'
                    , 'gestion.tablero15.nombreInstitucionEd'
                    , 'gestion.tablero15.codDanesede'
                    , 'gestion.tablero15.tipoSitio'
                    , 'gestion.tablero15.codDaneMuni'
                    , 'gestion.tablero15.nombre_centro_pob'
                    , 'gestion.tablero15.site_id'
                    , 'gestion.tablero15.matricula'
                    , 'gestion.tablero15.energiadesc'
                    , 'gestion.tablero15.grupoDesc'
                    , 'gestion.tablero15.cod_servicio'
                    , 'gestion.tablero15.region'
                    , 'gestion.tablero15.detalleSitio'
                    , 'gestion.tablero15.municipio'
                    #, 'gestion.tablero15.id_Beneficiario'
                    , 'gestion.tablero15.location'
                    , 'gestion.tablero15.codDaneInstitucionEdu'
                    , 'gestion.tablero15.categoria'
                    #, 'gestion.tablero15.opened_by'
                    , 'gestion.tablero15.subcategoria'
                    , 'gestion.tablero15.@timestamp'
                    , 'gestion.tablero15.fechaApertura'
                    , 'gestion.tablero15.id_Beneficiario'
                    , 'gestion.tablero15.category'
                    #, 'gestion.tablero15.fechaCierre'
                    , 'gestion.tablero15.canal'
                    , 'gestion.tablero15.numeroTicket'
                    , 'gestion.tablero15.total'
                    , 'gestion.tablero15.close_date'
                    #, 'gestion.tablero15.coderesolucion'
                    , 'gestion.tablero15.timeresolution'                                                                                       
                    , 'gestion.tablero15.callback_type'
                    , 'gestion.tablero15.initial_impact'
                    , 'gestion.tablero15.name'
                    , 'gestion.tablero15.variable2'
                    , 'gestion.tablero15.version'
                    , 'gestion.tablero15.contact_name'                                                                  
                    , 'gestion.tablero15.variable3'
                    , 'gestion.tablero15.anyo'
                    , 'gestion.tablero15.mes'
                    , 'gestion.tablero15.dia'
                  ]
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{str(document['gestion.tablero15.id_Beneficiario']) + '-' + str(document['gestion.tablero15.fechaApertura'])+ str(random.randrange(1000))}",
                "_source": filterKeys(document),
            }

In [22]:
def doc_ordenamiento(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": 1,
                "_source": filterKeys(document),
            }

In [23]:
datos_SM_interactions

""


In [43]:
datos_semilla

,gestion.tablero15.nombreSede,gestion.tablero15.DDA,gestion.tablero15.estadoInstalacion,gestion.tablero15.COD_ISO,gestion.tablero15.energia,gestion.tablero15.departamento,gestion.tablero15.codCentroPoblado,gestion.tablero15.nombreInstitucionEd,gestion.tablero15.codDanesede,gestion.tablero15.tipoSitio,...,gestion.tablero15.matricula,gestion.tablero15.energiadesc,gestion.tablero15.grupoDesc,gestion.tablero15.cod_servicio,gestion.tablero15.region,gestion.tablero15.detalleSitio,gestion.tablero15.municipio,gestion.tablero15.id_Beneficiario,gestion.tablero15.codDaneInstitucionEdu,gestion.tablero15.location
0,C.E.R LA PAULINA,5,3,CO-ANT,1,ANTIOQUIA,333,C. E. R. LA PAULINA,205854000501,1,...,111,RED INTERCONECTADA,OBLIGATORIOS,ZGYO176,A,8,VALDIVIA,22154,205854000501,"6.7048548,-76.14525469"
1,CENTRO EDUCATIVO RURAL EL CERRO,5,3,CO-ANT,1,ANTIOQUIA,333,I. E. MANUEL ANTONIO TORO,205284000201,1,...,22,RED INTERCONECTADA,OBLIGATORIOS,ZGYO177,A,8,FRONTINO,20022,105284000214,"4.225166667,-73.10716667"
2,C. E. R. MERCEDES YEPES,5,1,CO-ANT,1,ANTIOQUIA,333,C. E. R. EL CARMIN,205040000161,1,...,42,RED INTERCONECTADA,OBLIGATORIOS,,A,8,ANORÍ,18787,205040000357,"6.93848274,-75.151177190"
3,C. E. R. MEDIAS FALDAS,5,1,CO-ANT,4,ANTIOQUIA,333,I. E. ANORI,205040000616,1,...,84,TIENE ENERGÍA,Opción 2,,A,8,ANORÍ,18802,105040000212,"7.04797951,-75.19442611"
4,C. E. R. ROSAURA ORTEGA,5,2,CO-ANT,1,ANTIOQUIA,333,C. E. R. EL CARMIN,205040000641,1,...,14,RED INTERCONECTADA,OBLIGATORIOS,ZGYO583,A,8,ANORÍ,18804,205040000357,"7.06272584,-75.22730972"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6942,ESCUELA SANTA ISABEL,5,1,CO-VAU,3,VAUPÉS,333,I.E.D. DE PIEDRA ÑI,297001002239,1,...,53,SOLUCIÓN SOLAR,OBLIGATORIOS,,A,8,PACOA,67733,297001001879,"6.39026335,-73.138349330"
6943,ESCUELA RURAL DE SAN LUIS DE PIEDRA ÑI,5,1,CO-VAU,4,VAUPÉS,333,I.E.D. DE PIEDRA ÑI,297511000036,1,...,14,SIN INFORMACIÓN,OBLIGATORIOS,,A,8,PACOA,70622,297001001879,"0.08164385,-70.07416384"
6944,ESCUELA RURAL DE SANTA ROSA,5,1,CO-VAU,4,VAUPÉS,333,I.E.D. DE PIEDRA ÑI,297511000044,1,...,36,SIN INFORMACIÓN,OBLIGATORIOS,,A,8,PACOA,70623,297001001879,"4.596389,-74.074639"
6945,ESCUELA RURAL DE TOAKA,5,1,CO-VAU,4,VAUPÉS,333,I.E.D. DE PIEDRA ÑI,297511000028,1,...,32,SIN INFORMACIÓN,OBLIGATORIOS,,A,8,PACOA,70624,297001001879,"1.007644476,-71.25103723"


In [44]:
datos_SM_interactions.dtypes

Series([], dtype: object)

In [47]:
try:
    if not datos_SM_interactions.empty:
        concat = pd.merge(datos_SM_interactions,datos_semilla, on=['gestion.tablero15.id_Beneficiario'],how='inner')
        concat['gestion.tablero15.fechaApertura'] = concat['gestion.tablero15.fechaApertura'].str.replace("T"," ")
        concat['gestion.tablero15.fechaApertura'] = concat['gestion.tablero15.fechaApertura'].str.slice(stop=19)
        concat["gestion.tablero15.anyo"] = concat["gestion.tablero15.fechaApertura"].str[0:4]
        concat["gestion.tablero15.mes"] = concat["gestion.tablero15.fechaApertura"].str[5:7]
        concat["gestion.tablero15.dia"] = concat["gestion.tablero15.fechaApertura"].str[8:10]
        concat['@timestamp'] = now.isoformat()
        concat = concat.fillna('null')

        #salida = helpers.bulk(es, doc_generator(concat))
        #print("Fecha: ", now,"- Interacciones insertadas en indice principal:",salida[0])
except Exception as e:
    print(e, "\nNinguna Interacción insertada en indice principal:")

    

Actualizando fecha de control de ejecución

In [41]:
fecha_ejecucion = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)).strftime("%Y-%m-%d"'T'"%H:%M:%S")[0:15] + '0:00'    
#fecha_ejecucion = '2021-04-29T00:00:00'

if fecha_ejecucion > str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))[0:15] + '0:00':
    fecha_ejecucion = str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))[0:15] + '0:00'
response = es.index(
        index = indice_control,
        id = 'jerarquia-tablero15',
        body = { 'jerarquia-tablero15': 'jerarquia-tablero15','tablero15.fechaControl' : fecha_ejecucion}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion)



actualizada fecha control de ejecucion: 2021-08-15T01:00:00
